# Import

In [3]:
import os
import glob
import pandas as pd
#from multiprocessing import Pool

import bed_lib as bl
import div_trans_lib as dt
import basics_counts as bc

%matplotlib inline

In [2]:
# To not have to restart the kernel everytime changes are made in modules
%load_ext autoreload
%autoreload 2

**WARNINGS**

- TRY filtering only uniquely mapping reads (MAPQ 255) and compare with filtering only primary 

**TO EXPLAIN**

There is a rationale being only using reads (and not fragments), since we are only interested in ends of - reads first and then beginnings of + reads 
> Actually might be interesting to try using only one reads of the pair (TO CHECK)

## Input

In [8]:
bams = glob.glob('../data/bams/*_coordSort_filt.bam')
#bams = glob.glob('../data/bams/test/*.bam')
bams

['../data/bams/VEUDE_A_1_coordSort_filt.bam',
 '../data/bams/Compoud_A_1_coordSort_filt.bam']

# Analyse from bams

The idea here is to detect divergent transcription directly from a sorted bam.
This would fix some limitations of the first method.

ALGO: 
Look through the coordinated sorted bam file, registering the transcribed strand for each reads.

Since we have paired strand-specific libraries (rf):
- first of the pair, reverse complemented = + strand
- second of the pair, NOT reverse complemented = + strand
- first of the pair, NOT reverse complemented = - strand
- second of the pair, reverse complemented = - strand

When a change of sign is detected, calculate the distance with the previous transcript on the opposite strand 

## Detect divergent transcription events accross sorted bam

In [9]:
%%time
div_trans_beds = [dt.get_divergent_transcription_beds(bam, distance=500) for bam in bams]

2018-08-10 17:08:15,422 -  - div_trans_lib - get_divergent_transcription_beds() - INFO - Using already available bed file: bed_outfolder/VEUDE_A_1_coordSort_filt_divtrans.bed
2018-08-10 17:08:15,435 -  - div_trans_lib - get_divergent_transcription_beds() - INFO - Using already available bed file: bed_outfolder/Compoud_A_1_coordSort_filt_divtrans.bed


CPU times: user 6.43 ms, sys: 9.46 ms, total: 15.9 ms
Wall time: 18.1 ms


Merge the divergent transcription events detected:

In [10]:
div_trans_beds_merged = [bed.merge() for bed in div_trans_beds]

In [6]:
div_trans_beds_merged[0].head()

1	14627	14825
1	15143	15308
1	15834	15865
1	15919	16165
1	17432	17581
1	17860	17997
1	19182	19393
1	23683	23711
1	29246	29560
1	123748	123791


# Filter by overlap with CAGE TSS

Use FANTOM5 transcription initiation information obtained with CAGE data (either simple or divergent transcription) to filter divergent transcription events detected in RNA-seq alignments.

## Download cage peaks

In [ ]:
%%bash

wget http://fantom.gsc.riken.jp/5/datafiles/latest/extra/CAGE_peaks/hg19.cage_peak_phase1and2combined_coord.bed.gz \
    -O ../data/cage_peaks.bed.gz
    
gunzip ../data/cage_peaks.bed.gz
    
# Convert to Ensembl type bed
sed 's/^chr//g' ../data/cage_peaks.bed | bedtools sort -i - > ../data/cage_peaks_clean.bed

# Separate two beds (one per strand) to later catch both cage transcription direction
#awk '$6~"+" {print}' ../data/cage_peaks_clean.bed > ../data/cage_peaks_plusStrand.bed
#awk '$6~"-" {print}' ../data/cage_peaks_clean.bed > ../data/cage_peaks_minusStrand.bed

# Cleanup
rm ../data/cage_peaks.bed
#rm ../data/cage_peaks_clean.bed

## Filter divergent transcription events with CAGE dataset
### Detect divergent transcription in CAGE dataset

In [7]:
cage_bed = bl.Bed('../data/cage_peaks_clean.bed')
cage_div_trans_out = '../data/cage_div_trans.bed'

with open(cage_div_trans_out, 'w') as f:
        for interval in dt.identify_divergent_transcription(cage_bed.get_intervals()):
            f.write('\t'.join([str(x) for x in interval]) + '\n')
        
cage_div_trans_bed = bl.Bed(cage_div_trans_out)

### Select RNA-seq divtrans events overlapping with CAGE divtrans events

In [8]:
div_trans_intersect_cage = [bed.intersect(cage_div_trans_bed, supp_args='-wa')
                                for bed in div_trans_beds_merged]

In [9]:
pd.DataFrame([bed.stats() for bed in div_trans_intersect_cage])

,Nbases,Nintervals,bedobj,name,path
0,911750,2034,<Bed object: VEUDE_A_1_coordSort_filt_divtrans...,VEUDE_A_1_coordSort_filt_divtrans_M-inter-cage...,bed_outfolder/VEUDE_A_1_coordSort_filt_divtran...
1,1065836,2106,<Bed object: Compoud_A_1_coordSort_filt_divtra...,Compoud_A_1_coordSort_filt_divtrans_M-inter-ca...,bed_outfolder/Compoud_A_1_coordSort_filt_divtr...


# Filter by FANTOM5 CAGE enhancer prediction [SLIDEBASE]

One can use SLIDEBASE for enhancer http://slidebase.binf.ku.dk/human_enhancers/results

Downloading bed from SLIDEBASE (same as SLIDEBASE "Download BED"):

In [ ]:
%%bash
    
wget http://slidebase.binf.ku.dk/human_enhancers/bed -O ../data/slidebase_enhancers.bed

# Convert to Ensembl type bed
sed 's/^chr//g' ../data/slidebase_enhancers.bed | bedtools sort -i - > ../data/slidebase_enhancers_clean.bed

rm ../data/slidebase_enhancers.bed

In [10]:
slidebase_enh = bl.Bed('../data/slidebase_enhancers_clean.bed')

In [11]:
div_trans_intersect_slidebase = [bed.intersect(slidebase_enh, supp_args='-wa')
                                for bed in div_trans_beds_merged]

In [12]:
pd.DataFrame([bed.stats() for bed in div_trans_intersect_slidebase])

,Nbases,Nintervals,bedobj,name,path
0,1152057,3191,<Bed object: VEUDE_A_1_coordSort_filt_divtrans...,VEUDE_A_1_coordSort_filt_divtrans_M-inter-slid...,bed_outfolder/VEUDE_A_1_coordSort_filt_divtran...
1,1597962,3695,<Bed object: Compoud_A_1_coordSort_filt_divtra...,Compoud_A_1_coordSort_filt_divtrans_M-inter-sl...,bed_outfolder/Compoud_A_1_coordSort_filt_divtr...


# Filter by Transcription factors intervals

Download transcription factors bed from UCSC:

**IMPORTANT NOTE:** At the difference of enhancer and div_trans intervals, transcription factors intervals are not merged 
> This leads to divergent estimations in intersection when considering jaccard and intersect results.


**WARNING** The download took ~1 hour 

In [42]:
%%bash

# Download from UCSC mysql database, remove the 'chr',the header, sort the bed
mysql --user=genome --host=genome-mysql.cse.ucsc.edu -A -e "use hg19; SELECT * FROM wgEncodeRegTfbsClusteredV3;" | \
    awk -v OFS='\t' 'NR>1 {gsub("chr","",$2); print $2, $3, $4, $5, $6}' | \
    bedtools sort -i - > ../data/wgEncodeRegTfbsClusteredV3.bed

In [13]:
transcription_factors_bed = bl.Bed('../data/wgEncodeRegTfbsClusteredV3.bed').merge()

In [14]:
div_trans_intersect_transFact = [bed.intersect(transcription_factors_bed, supp_args='-wa')
                                for bed in div_trans_beds_merged]

In [15]:
pd.DataFrame([bed.stats() for bed in div_trans_intersect_transFact])

,Nbases,Nintervals,bedobj,name,path
0,29606523,89167,<Bed object: VEUDE_A_1_coordSort_filt_divtrans...,VEUDE_A_1_coordSort_filt_divtrans_M-inter-wgEn...,bed_outfolder/VEUDE_A_1_coordSort_filt_divtran...
1,32938330,94972,<Bed object: Compoud_A_1_coordSort_filt_divtra...,Compoud_A_1_coordSort_filt_divtrans_M-inter-wg...,bed_outfolder/Compoud_A_1_coordSort_filt_divtr...


# Filter by Chromatin state infered enhancers

Download epigenomes data from the NIH Roadmap Epigenomics Mapping Consortium 

see: https://egg2.wustl.edu/roadmap/web_portal/chr_state_learning.html#core_15state

**WARNING:** ~45min download

In [9]:
%%bash
#wget -P ../data/chmm_enhancers http://egg2.wustl.edu/roadmap/data/byFileType/chromhmmSegmentations/ChmmModels/coreMarks/jointModel/final/all.mnemonics.bedFiles.tgz

tar -C ../data/chmm_enhancers/ -zxvf ../data/chmm_enhancers/all.mnemonics.bedFiles.tgz
gunzip ../data/chmm_enhancers/*.gz

E001_15_coreMarks_mnemonics.bed.gz
E002_15_coreMarks_mnemonics.bed.gz
E003_15_coreMarks_mnemonics.bed.gz
E004_15_coreMarks_mnemonics.bed.gz
E005_15_coreMarks_mnemonics.bed.gz
E006_15_coreMarks_mnemonics.bed.gz
E007_15_coreMarks_mnemonics.bed.gz
E008_15_coreMarks_mnemonics.bed.gz
E009_15_coreMarks_mnemonics.bed.gz
E010_15_coreMarks_mnemonics.bed.gz
E011_15_coreMarks_mnemonics.bed.gz
E012_15_coreMarks_mnemonics.bed.gz
E013_15_coreMarks_mnemonics.bed.gz
E014_15_coreMarks_mnemonics.bed.gz
E015_15_coreMarks_mnemonics.bed.gz
E016_15_coreMarks_mnemonics.bed.gz
E017_15_coreMarks_mnemonics.bed.gz
E018_15_coreMarks_mnemonics.bed.gz
E019_15_coreMarks_mnemonics.bed.gz
E020_15_coreMarks_mnemonics.bed.gz
E021_15_coreMarks_mnemonics.bed.gz
E022_15_coreMarks_mnemonics.bed.gz
E023_15_coreMarks_mnemonics.bed.gz
E024_15_coreMarks_mnemonics.bed.gz
E025_15_coreMarks_mnemonics.bed.gz
E026_15_coreMarks_mnemonics.bed.gz
E027_15_coreMarks_mnemonics.bed.gz
E028_15_coreMarks_mnemonics.bed.gz
E029_15_coreMarks_mn

Add cell line information to the bed before merging. Here it could be maybe interesting to merge according to a percentage of overlap (not overlapping intervals with only 1 base overlap for example)

The mnemonics selected are:

* 1_TssA
* 2_TssAFlnk
* 7_Enh
* 6_EnhG
* 10_TssBiv
* 12_EnhBiv

In [5]:
%%bash

cd ../data/chmm_enhancers/

# Add the name of the cell line to bed annotations:
for i in *.bed; do awk -v cellline=${i%%_*} 'BEGIN{OFS="\t"}; ($4~/Enh|Tss/) {print $1, $2, $3, $4 "_" cellline}' $i > ${i%%.*}_tmp.bed; done
# Concatenate all Enh files, bedtools sort / merge / remove 'chr'
cat E*tmp.bed | sort -k1,1 -k2,2n | bedtools merge  -c 4 -o collapse | sed 's/^chr//g' > cat_all_ENh_Tss_all_tissu_sort_merged.bed

In [6]:
epigenomes_bed = bl.Bed('../data/chmm_enhancers/cat_all_ENh_Tss_all_tissu_sort_merged.bed')

In [11]:
div_trans_intersect_epigenomes = [bed.intersect(epigenomes_bed, supp_args='-wa') 
                                  for bed in div_trans_beds_merged]

In [ ]:
[x.stats() for x in div_trans_intersect_epigenomes]